In [ ]:
import hashlib
import os
import requests
import zipfile
from libretranslatepy import LibreTranslateAPI
import pandas as pd

# Настройки
dataset_url = "https://object.pouta.csc.fi/OPUS-PHP/v1/moses/en-ru.txt.zip"
dataset_zip = "en-ru.txt.zip"
en_file = "PHP.en-ru.en"
ru_file = "PHP.en-ru.ru"
cache_dir = "./docs/php/cache"
output_file = "./datasets/docs_php.xlsx"

# Шаг 1: Скачивание и разархивирование датасета
if not os.path.exists(dataset_zip):
    response = requests.get(dataset_url)
    with open(dataset_zip, 'wb') as f:
        f.write(response.content)

with zipfile.ZipFile(dataset_zip, 'r') as zip_ref:
    zip_ref.extractall()

# Шаг 2: Чтение строк из файла PHP.en-ru.en
with open(en_file, 'r', encoding='utf-8') as f:
    en_lines = f.readlines()

# Создание директории для кеша, если она не существует
os.makedirs(cache_dir, exist_ok=True)

# Шаг 3: Перевод строк с использованием LibreTranslate и кеширование
lt = LibreTranslateAPI("http://localhost:5000")

def translate_and_cache(text, src_lang='en', tgt_lang='ru'):
    # Создание хеша для строки
    text_hash = hashlib.md5(text.encode('utf-8')).hexdigest()
    cache_file = os.path.join(cache_dir, f"{text_hash}.txt")
    
    # Проверка, существует ли уже переведенная строка в кеше
    if os.path.exists(cache_file):
        with open(cache_file, 'r', encoding='utf-8') as f:
            translation = f.read()
    else:
        translation = lt.translate(text, src_lang, tgt_lang)
        with open(cache_file, 'w', encoding='utf-8') as f:
            f.write(translation)
    
    return translation

# Перевод всех строк и сохранение в список
en_lines = [line.strip() for line in en_lines]
ru_lines = [translate_and_cache(line.strip()) for line in en_lines]

# Шаг 4: Создание параллельного корпуса и сохранение его в формате LibreOffice Calc
os.makedirs(os.path.dirname(output_file), exist_ok=True)
df = pd.DataFrame({'English': en_lines, 'Russian': ru_lines})
df = df.drop_duplicates()

df.to_excel(output_file, index=False)

print(f"Parallel corpora saved to {output_file}")